In [1]:
import csv
import re
import tiktoken
import json

In [2]:
tokenizer = tiktoken.get_encoding('gpt2')

In [3]:
symptom_vocabulary = set()
try:
    with open('health_dataset.csv', 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        for row in reader:
            for phrase in row:
                cleaned_phrase = phrase.strip().lower()
                if cleaned_phrase:
                    symptom_vocabulary.add(cleaned_phrase)
    print(f" Loaded {len(symptom_vocabulary)} symptom phrases.")
    print("-" * 30)
except FileNotFoundError:
    print(" ERROR: health_dataset.csv not found!")
    exit()
except Exception as e:
    print(f" An error occurred reading the CSV: {e}")
    exit()


 Loaded 145 symptom phrases.
------------------------------


In [4]:
user_query = input("Hey! What's on your mind today: ")
user_query_lower = user_query.lower()

In [5]:
matched_symptoms_strings = set()
for symptom_phrase in symptom_vocabulary:
    if symptom_phrase in user_query_lower:
        matched_symptoms_strings.add(symptom_phrase)

if not matched_symptoms_strings:
    print("\nNo symptoms from the vocabulary were found in the query.")
else:
    print(f"\n Matched Symptoms (as strings): {matched_symptoms_strings}")


 Matched Symptoms (as strings): {'pain'}


In [6]:
symptom_token_map = {}
if matched_symptoms_strings:
    print("\n" + "="*40)
    print("Symptom Token ID Analysis")
    print("="*40)
    for symptom in matched_symptoms_strings:
        tokens = tokenizer.encode(symptom)
        symptom_token_map[symptom] = tokens
        print(f"'{symptom}': {tokens}")



Symptom Token ID Analysis
'pain': [35436]


In [7]:
print("\n" + "="*40)
print("🩺 Finding Potential Treatments...")
print("="*40)


🩺 Finding Potential Treatments...


In [8]:

if not matched_symptoms_strings:
    print("No symptoms were matched, cannot suggest treatment.")
else:
    try:
        with open('updatedSolution.json', 'r', encoding='utf-8') as sol:
            disease_database = json.load(sol)

        found_match = False

        def check_disease(disease_entry):
            symptom_list = disease_entry.get('symptoms')
            if not symptom_list:
                return False

            json_symptoms = set(s.strip().lower() for s in symptom_list)

            overlapping_symptoms = matched_symptoms_strings.intersection(json_symptoms)

            if overlapping_symptoms:
                disease_name = disease_entry.get('disease', 'Unknown Disease')
                aid_1 = disease_entry.get('first_aid_treatment_1', 'No first aid 1 listed.')
                aid_2 = disease_entry.get('first_aid_treatment_2', 'No first aid 2 listed.')

                print(f"\n--- Possible Match Found ---")
                print(f"Disease: {disease_name}")
                # This line is added for more clarity
                print(f"Matched Symptoms: {list(overlapping_symptoms)}")
                print(f"  First Aid 1: {aid_1}")
                print(f"  First Aid 2: {aid_2}")
                return True

            return False


        if isinstance(disease_database, list):
            for disease_entry in disease_database:
                if check_disease(disease_entry):
                    found_match = True


        elif isinstance(disease_database, dict):

            disease_list = disease_database.get('data')

            if isinstance(disease_list, list):
                for disease_entry in disease_list:
                    if check_disease(disease_entry):
                        found_match = True
            else:
                print(" ERROR: JSON is a dictionary but has no 'data' key with a list.")

        if not found_match:
            print("\nYour symptoms did not match a specific disease in the database.")

    except FileNotFoundError:
        print(" ERROR: solution.json not found!")
    except json.JSONDecodeError:
        print(" ERROR: solution.json is not a valid JSON file.")
    except Exception as e:
        print(f" An error occurred reading or processing solution.json: {e}")


--- Possible Match Found ---
Disease: Impetigo
Matched Symptoms: ['pain']
  First Aid 1: Wash sores gently with antibacterial soap and warm water 2-3 times daily, soaking off any crusts; pat dry with clean towel
  First Aid 2: Apply prescribed antibiotic ointment or cream (mupirocin) 2-3 times daily and cover with waterproof bandage to prevent spreading; complete full antibiotic course
